In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. 导入必要库，选择场类型

In [72]:
import torch
import os
import sys
import hydra
import traceback
from pathlib import Path
from lightning import Fabric
from torch_geometric.utils import to_dense_batch

# 设置项目路径，导入项目相关模块
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))
from funcmol.utils.constants import PADDING_INDEX
from funcmol.utils.utils_nf import load_neural_field, get_latest_model_path
from funcmol.dataset.dataset_field import create_field_loaders, create_gnf_converter
from gnf_visualizer import visualize_reconstruction

# 设置 torch.compile 兼容性
import torch._dynamo
torch._dynamo.config.suppress_errors = True

## 2. 完成重建过程

In [73]:
fabric = Fabric(
    accelerator="auto",
    devices=1,
    precision="32-true",
    strategy="auto"
)
fabric.launch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

config_dir = str(Path.cwd().parent / "configs")
with hydra.initialize_config_dir(config_dir=config_dir, version_base=None):
    config = hydra.compose(config_name="train_nf_qm9")

# 在这里修改数据路径
config["dset"]["data_dir"] = str(Path.cwd().parent / "dataset" / "data")

# 查找最新的模型文件
# 使用工具函数查找最新的模型文件
model_path = get_latest_model_path(
    base_exp_dir=str(Path.cwd().parent / "exps" / "neural_field"),
    model_prefix="nf_qm9_"
)
# TODO：可以在这里手动修改模型路径
model_path = "/home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250801_223228_271628"

print(f"Loading model from: {model_path}")

model_path = Path(model_path) / "model.pt"

# 加载训练好的权重
checkpoint = fabric.load(str(model_path))
enc = load_neural_field(checkpoint, fabric, config)[0]
dec = load_neural_field(checkpoint, fabric, config)[1]

print("Model loaded successfully!")

enc = enc.to(device)
dec = dec.to(device)

# 创建GNFConverter实例用于数据加载
converter = create_gnf_converter(config, device="cpu")

loader_val = create_field_loaders(config, converter, split="val", fabric=fabric)
batch = next(iter(loader_val)).to(device)
coords, _ = to_dense_batch(batch.pos, batch.batch, fill_value=0)
atoms_channel, _ = to_dense_batch(batch.x, batch.batch, fill_value=PADDING_INDEX)
gt_coords = coords
gt_types = atoms_channel

with torch.no_grad():
    codes = enc(batch)

output_dir = Path.cwd() / "gnf_visualization_outputs"
output_dir.mkdir(exist_ok=True)

Using latest model from: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250801_230159_641094
Loading model from: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250801_223228_271628


>> loaded dec
>> loaded enc
>> loaded dec
>> loaded enc
Model loaded successfully!
>> val set size: 20042


In [74]:
results = visualize_reconstruction(
    gt_coords, gt_types, converter, dec, codes,
    output_dir=str(output_dir),
    sample_idx=1, # 定义要分析的样本索引
    field_option="gt_field" # TODO：可以在这里修改使用的场的类型
)


Starting reconstruction for molecule 1
Ground truth atoms: 9
Atom types: [0, 2, 0, 1, 1, 1, 1, 1, 1]
Iteration 0, Atom type 0: grad norm = 0.021045
Iteration 0, Atom type 1: grad norm = 0.036422
Iteration 0, Atom type 2: grad norm = 0.009120
Iteration 0, Atom type 3: grad norm = 0.000000
Iteration 0, Atom type 4: grad norm = 0.000000
Iteration 1000, Atom type 0: grad norm = 0.001361
Iteration 1000, Atom type 1: grad norm = 0.008513
Iteration 1000, Atom type 2: grad norm = 0.000175
Iteration 1000, Atom type 3: grad norm = 0.000000
Iteration 1000, Atom type 4: grad norm = 0.000000
[DBSCAN] Total points: 500, Clusters found: 2, Noise points: 475
[DBSCAN] Total points: 500, Clusters found: 3, Noise points: 466
[DBSCAN] Total points: 500, Clusters found: 1, Noise points: 481
[DBSCAN] Total points: 500, Clusters found: 0, Noise points: 500
[DBSCAN] Total points: 500, Clusters found: 0, Noise points: 500


## 3. 可视化一维梯度场

In [75]:
from funcmol.gnf_visualizer import visualize_1d_gradient_field_comparison

# 创建output文件夹
output_dir = Path.cwd() / "gnf_visualization_outputs"
output_dir.mkdir(exist_ok=True)

# 添加梯度场分析
print("\n" + "="*50)
print("开始梯度场分析...")

# 定义要分析的样本索引
sample_idx = 1

# 检查样本中是否有不同类型的原子
gt_mask = (gt_types[sample_idx] != PADDING_INDEX)
gt_valid_types = gt_types[sample_idx][gt_mask]

# 为每种原子类型进行梯度场分析
for atom_type in range(5):  # 0=C, 1=H, 2=O, 3=N, 4=F
    if (gt_valid_types == atom_type).sum() > 0:
        print(f"\n分析 {['C', 'H', 'O', 'N', 'F'][atom_type]} 原子的梯度场...")
        
        try:
            # 使用分子质心作为y,z坐标
            molecule_center = gt_coords[sample_idx][gt_mask].mean(dim=0)
            
            # 修改保存路径到output文件夹
            save_path = output_dir / f"gradient_field_sample_{sample_idx}_{['C', 'H', 'O', 'N', 'F'][atom_type]}.png"
            
            result = visualize_1d_gradient_field_comparison(
                gt_coords=gt_coords,
                gt_types=gt_types,
                converter=converter,
                decoder=dec,
                codes=codes,
                sample_idx=sample_idx, 
                atom_type=atom_type,
                y_coord=molecule_center[1].item(),
                z_coord=molecule_center[2].item(),
                save_path=str(save_path)  # 转换为字符串
            )
            
            if result:
                print(f"{['C', 'H', 'O', 'N', 'F'][atom_type]} 原子梯度场分析完成！")
                print(f"MSE: {result['mse']:.6f}, MAE: {result['mae']:.6f}")
            else:
                print(f"{['C', 'H', 'O', 'N', 'F'][atom_type]} 原子梯度场分析失败")
                
        except Exception as e:
            print(f"分析 {['C', 'H', 'O', 'N', 'F'][atom_type]} 原子时出错: {e}")
            traceback.print_exc()
    else:
        print(f"样本中没有 {['C', 'H', 'O', 'N', 'F'][atom_type]} 原子，跳过分析")

print("\n所有分析完成！")


开始梯度场分析...

分析 C 原子的梯度场...
Gradient field comparison saved to: /home/huayuchen/funcmol-main-neuralfield/funcmol/notebooks/gnf_visualization_outputs/gradient_field_sample_1_C.png
C 原子梯度场分析完成！
MSE: 0.021815, MAE: 0.060471

分析 H 原子的梯度场...
Gradient field comparison saved to: /home/huayuchen/funcmol-main-neuralfield/funcmol/notebooks/gnf_visualization_outputs/gradient_field_sample_1_H.png
H 原子梯度场分析完成！
MSE: 0.029804, MAE: 0.084364

分析 O 原子的梯度场...
Gradient field comparison saved to: /home/huayuchen/funcmol-main-neuralfield/funcmol/notebooks/gnf_visualization_outputs/gradient_field_sample_1_O.png
O 原子梯度场分析完成！
MSE: 0.003080, MAE: 0.026402
样本中没有 N 原子，跳过分析
样本中没有 F 原子，跳过分析

所有分析完成！
